In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'NanumSquare'

In [2]:
users = pd.read_csv('./data/users.csv') # subsr, subsr_id, 사용자의 kids 유무
vods = pd.read_csv('./data/vods.csv', dtype={'program_id':str}) # vod 로그 기록 (분석용으로 추천함)
conts = pd.read_csv('./data/conts.csv', dtype={'program_id':str}) # contents 로그 기록
vods_sumut = pd.read_csv('./data/vods_sumut.csv', dtype={'program_id':str}) # vod 로그 기록 - subsr, program_id, 날짜 기준으로 중복 데이터 합친 것 (모델링용으로 추천함)
program_info_all = pd.read_csv('./data/program_data/program_info_all.csv', dtype={'program_id':str}) # cont, vod 합쳐진 것

In [279]:
# e_bool == 0 인 데이터만 뽑기
vod_log = vods_sumut[vods_sumut['e_bool']==0][['subsr_id', 'program_id', 'program_name', 'episode_num', 'log_dt', 'use_tms', 'disp_rtm_sec', 'count_watch']]
cont_log = conts[conts['e_bool']==0][['subsr_id', 'program_id', 'program_name', 'episode_num', 'log_dt']]
vod_info = program_info_all[program_info_all['e_bool']==0][['program_id','program_name', 'ct_cl', 'program_genre', 'ACTR_DISP', 'age_limit']]
#user_info = users.copy()

### 시간대/기간별 분석

In [17]:
vods_sumut['week'] = pd.to_datetime(vods_sumut['log_dt']).dt.week
conts['week'] = pd.to_datetime(conts['log_dt']).dt.week

In [50]:
vods_sumut['log_dt'] = pd.to_datetime(vods_sumut['log_dt'])
conts['log_dt'] = pd.to_datetime(conts['log_dt'])

In [60]:
# 휴일여부
vods_sumut.loc[vods_sumut[vods_sumut['log_dt'].dt.strftime('%Y-%m-%d') == '2023-09-28'].index, 'isholiday'] = 1
vods_sumut.loc[vods_sumut[vods_sumut['log_dt'].dt.strftime('%Y-%m-%d') == '2023-09-29'].index, 'isholiday'] = 1
vods_sumut.loc[vods_sumut[vods_sumut['log_dt'].dt.strftime('%Y-%m-%d') == '2023-09-30'].index, 'isholiday'] = 1
vods_sumut.loc[vods_sumut[vods_sumut['log_dt'].dt.strftime('%Y-%m-%d') == '2023-08-15'].index, 'isholiday'] = 1

In [61]:
conts.loc[conts[conts['log_dt'].dt.strftime('%Y-%m-%d') == '2023-09-28'].index, 'isholiday'] = 1
conts.loc[conts[conts['log_dt'].dt.strftime('%Y-%m-%d') == '2023-09-29'].index, 'isholiday'] = 1
conts.loc[conts[conts['log_dt'].dt.strftime('%Y-%m-%d') == '2023-09-30'].index, 'isholiday'] = 1
conts.loc[conts[conts['log_dt'].dt.strftime('%Y-%m-%d') == '2023-08-15'].index, 'isholiday'] = 1

In [63]:
conts['isholiday'] = conts['isholiday'].fillna(0)
vods_sumut['isholiday'] = vods_sumut['isholiday'].fillna(0)

In [66]:
# 주중 / 주말 여부
vods_sumut.loc[vods_sumut[vods_sumut['day_name'].isin(['Saturday', 'Sunday'])].index, 'isweekend'] = 1
conts.loc[conts[conts['day_name'].isin(['Saturday', 'Sunday'])].index, 'isweekend'] = 1
vods_sumut['isweekend'] = vods_sumut['isweekend'].fillna(0)
conts['isweekend'] = conts['isweekend'].fillna(0)

In [296]:
vod_log = vods_sumut[vods_sumut['e_bool']==0][['subsr_id', 'program_id', 'program_name', 'episode_num', 'ct_cl', 'program_genre', 'log_dt', 'month', 'week', 'day', 'hour', 'minute', 'weekday', 'day_name', 'isholiday', 'isweekend', 'age_limit', 'use_tms', 'disp_rtm_sec', 'count_watch']]
cont_log = conts[conts['e_bool']==0][['subsr_id', 'program_id', 'program_name', 'episode_num', 'ct_cl', 'program_genre', 'log_dt', 'month', 'week', 'day', 'hour', 'minute', 'weekday', 'day_name', 'isholiday', 'isweekend', 'age_limit']]

In [75]:
vod_log.to_csv('./EDA/vod_log.csv', encoding='cp949')
cont_log.to_csv('./EDA/cont_log.csv', encoding='cp949')

#### 월별 VOD 이용 추이

In [92]:
# 월별 방문 user 수 (unique)
monthly = cont_log.groupby(['month', 'subsr_id']).count()[['program_id']].groupby('month').count().reset_index().rename(columns={'program_id':'방문user수'})
monthly

,month,방문user수
0,8,645
1,9,649


In [93]:
# 월별 시청 user수 (unique)
monthly = monthly.merge(vod_log.groupby(['month', 'subsr_id']).count()[['program_id']].groupby('month').count().reset_index().rename(columns={'program_id':'시청user수'}), how='left')

In [94]:
# 월별 방문수
monthly = monthly.merge(cont_log.groupby(['month']).count()[['program_id']].reset_index().rename(columns={'program_id':'상세보기조회수'}), how='left')

In [95]:
# 월별 시청수
monthly = monthly.merge(vod_log.groupby(['month']).count()[['program_id']].reset_index().rename(columns={'program_id':'시청수'}), how='left')

In [96]:
monthly

,month,방문user수,시청user수,상세보기조회수,시청수
0,8,645,318,15572,4018
1,9,649,283,14285,3683


In [157]:
monthly.to_csv('./EDA/monthly.csv')

#### 주별 VOD 이용 추이

In [98]:
# 주별 방문 user 수 (unique)
weekly = cont_log.groupby(['week', 'subsr_id']).count()[['program_id']].groupby('week').count().reset_index().rename(columns={'program_id':'방문user수'})
weekly

,week,방문user수
0,31,299
1,32,284
2,33,273
3,34,244
4,35,295
5,36,282
6,37,278
7,38,263
8,39,283


In [102]:
# 주별 시청 user수 (unique)
weekly = weekly.merge(vod_log.groupby(['week', 'subsr_id']).count()[['program_id']].groupby('week').count().reset_index().rename(columns={'program_id':'시청user수'}), how='left')

In [107]:
# 주별 상세보기 조회수
weekly = weekly.merge(cont_log.groupby(['week']).count()[['program_id']].reset_index().rename(columns={'program_id':'상세보기조회수'}), how='left')

In [108]:
# 주별 시청수
weekly = weekly.merge(vod_log.groupby(['week']).count()[['program_id']].reset_index().rename(columns={'program_id':'시청수'}), how='left')

In [158]:
weekly.to_csv('./EDA/weekly.csv')

In [260]:
weekly

,week,방문user수,시청user수,상세보기조회수,시청수
0,31,299,166,3729,971
1,32,284,139,2803,696
2,33,273,140,3541,860
3,34,244,116,3321,868
4,35,295,146,3802,1139
5,36,282,137,3598,966
6,37,278,140,2847,826
7,38,263,109,3267,752
8,39,283,135,2949,623


#### 요일별 VOD 이용 추이

In [117]:
# 요일별 하루 평균 방문 user수 (unique)
daily = cont_log.groupby(['week', 'day_name', 'subsr_id']).count()[['program_id']].groupby(['week', 'day_name']).count().groupby('day_name').mean().reset_index().rename(columns={'program_id':'평균방문user수'})
daily

,day_name,평균방문user수
0,Friday,76.888889
1,Monday,70.500000
2,Saturday,85.888889
3,Sunday,80.250000
4,Thursday,72.000000
5,Tuesday,72.555556
6,Wednesday,73.222222


In [119]:
# 요일별 하루 평균 시청user수 (unique)
daily = daily.merge(vod_log.groupby(['week','day_name', 'subsr_id']).count()[['program_id']].groupby(['week', 'day_name']).count().groupby('day_name').mean().reset_index().rename(columns={'program_id':'평균시청user수'}), how='left')

In [126]:
# 요일별 평균상세보기조회수
daily = daily.merge(cont_log.groupby(['week', 'day_name']).count()[['program_id']].groupby('day_name').mean().reset_index().rename(columns={'program_id':'평균상세보기조회수'}), how='left')

In [128]:
# 요일별 평균시청수
daily = daily.merge(vod_log.groupby(['week', 'day_name']).count()[['program_id']].groupby('day_name').mean().reset_index().rename(columns={'program_id':'평균시청수'}), how='left')

In [159]:
daily.to_csv('./EDA/daily.csv')

In [261]:
daily

,day_name,평균방문user수,평균시청user수,평균상세보기조회수,평균시청수
0,Friday,76.888889,44.777778,577.888889,129.444444
1,Monday,70.500000,37.500000,452.750000,102.125000
2,Saturday,85.888889,45.444444,528.777778,131.333333
3,Sunday,80.250000,46.375000,438.500000,150.625000
4,Thursday,72.000000,45.777778,459.333333,125.111111
5,Tuesday,72.555556,41.555556,505.666667,120.000000
6,Wednesday,73.222222,43.444444,453.555556,125.111111


#### 시간대별 VOD 이용 추이

In [131]:
# 시간대별 시간 평균 방문user수 (unique)
hourly = cont_log.groupby(['day','hour','subsr_id']).count()[['program_id']].groupby(['day','hour']).count().groupby('hour').mean().reset_index().rename(columns={'program_id':'평균방문user수'})

In [136]:
# 시간대별 시간 평균 시청user 수 (unique)
hourly = hourly.merge(vod_log.groupby(['day','hour','subsr_id']).count()[['program_id']].groupby(['day','hour']).count().groupby('hour').mean().reset_index().rename(columns={'program_id':'평균시청user수'}), how='left')

In [144]:
# 시간대별 시간 평균 상세보기조회수
hourly = hourly.merge(cont_log.groupby(['day', 'hour']).count()[['program_id']].groupby('hour').mean().reset_index().rename(columns={'program_id':'평균상세보기조회수'}), how='left')

In [145]:
# 시간대별 시간 평균 시청수
hourly = hourly.merge(vod_log.groupby(['day', 'hour']).count()[['program_id']].groupby('hour').mean().reset_index().rename(columns={'program_id':'평균시청수'}), how='left')

In [160]:
hourly.to_csv('./EDA/hourly.csv')
hourly_주중.to_csv('./EDA/hourly_주중.csv')
hourly_주말.to_csv('./EDA/hourly_주말.csv')

In [151]:
# 주중
hourly_주중 = cont_log[cont_log['isweekend']==0].groupby(['day','hour','subsr_id']).count()[['program_id']].groupby(['day','hour']).count().groupby('hour').mean().reset_index().rename(columns={'program_id':'평균방문user수'})
hourly_주중 = hourly_주중.merge(vod_log[vod_log['isweekend']==0].groupby(['day','hour','subsr_id']).count()[['program_id']].groupby(['day','hour']).count().groupby('hour').mean().reset_index().rename(columns={'program_id':'평균시청user수'}), how='left')
hourly_주중 = hourly_주중.merge(cont_log[cont_log['isweekend']==0].groupby(['day', 'hour']).count()[['program_id']].groupby('hour').mean().reset_index().rename(columns={'program_id':'평균상세보기조회수'}), how='left')
hourly_주중 = hourly_주중.merge(vod_log[vod_log['isweekend']==0].groupby(['day', 'hour']).count()[['program_id']].groupby('hour').mean().reset_index().rename(columns={'program_id':'평균시청수'}), how='left')

In [153]:
# 주말
hourly_주말 = cont_log[cont_log['isweekend']==1].groupby(['day','hour','subsr_id']).count()[['program_id']].groupby(['day','hour']).count().groupby('hour').mean().reset_index().rename(columns={'program_id':'평균방문user수'})
hourly_주말 = hourly_주말.merge(vod_log[vod_log['isweekend']==1].groupby(['day','hour','subsr_id']).count()[['program_id']].groupby(['day','hour']).count().groupby('hour').mean().reset_index().rename(columns={'program_id':'평균시청user수'}), how='left')
hourly_주말 = hourly_주말.merge(cont_log[cont_log['isweekend']==1].groupby(['day', 'hour']).count()[['program_id']].groupby('hour').mean().reset_index().rename(columns={'program_id':'평균상세보기조회수'}), how='left')
hourly_주말 = hourly_주말.merge(vod_log[vod_log['isweekend']==1].groupby(['day', 'hour']).count()[['program_id']].groupby('hour').mean().reset_index().rename(columns={'program_id':'평균시청수'}), how='left')

In [358]:
hourly_주중.columns=['hour', '주중_평균방문user수', '주중_평균시청user수', '주중_평균상세보기조회수', '주중_평균시청수']

In [360]:
hourly_주말.columns=['hour', '주말_평균방문user수', '주말_평균시청user수', '주말_평균상세보기조회수', '주말_평균시청수']

In [363]:
hourly_주중.merge(hourly_주말, how='left').to_csv('./EDA/hourly_주중_주말.csv')

### 사용자별 분석

In [189]:
user_info = users.copy()

In [257]:
user_info

,subsr,kids,subsr_id,상세보기vod수,시청vod수,주평균시청vod수,방문빈도수,상세보기조회빈도(클릭수),평균 클릭수,시청빈도(시청수),평균 시청수,시청시간합계,평균 시청시간,vod평균시청율,시청전환율,청불여부_상세보기,청불여부_시청
0,65941000,0,0,4.0,4.0,1.666667,6.0,15.0,2.500000,21.0,3.500000,47634.0,7939.000000,0.858891,1.000000,0.0,0.0
1,66873000,0,1,45.0,25.0,7.000000,56.0,354.0,6.321429,128.0,2.285714,381103.0,6805.410714,0.622109,0.555556,1.0,0.0
2,61689000,0,2,11.0,1.0,1.000000,10.0,17.0,1.700000,2.0,0.200000,9240.0,924.000000,1.000000,0.090909,0.0,0.0
3,61619000,0,3,36.0,12.0,6.000000,5.0,117.0,23.400000,14.0,2.800000,56282.0,11256.400000,0.922184,0.333333,1.0,0.0
4,64495000,0,4,18.0,3.0,1.666667,11.0,115.0,10.454545,33.0,3.000000,100861.0,9169.181818,0.655634,0.166667,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,64650000,0,918,1.0,NaN,NaN,1.0,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
919,65379000,1,919,1.0,NaN,NaN,1.0,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
920,61592000,1,920,1.0,NaN,NaN,1.0,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
921,61620000,1,921,1.0,NaN,NaN,1.0,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [191]:
# 사용자별 상세보기 vod수
user_info = user_info.merge(cont_log.groupby(['subsr_id', 'program_id']).count()[['program_name']].groupby('subsr_id').count().reset_index().rename(columns={'program_name':'상세보기vod수'}), how='left')

In [192]:
# 사용자별 시청 vod수
user_info = user_info.merge(vod_log.groupby(['subsr_id', 'program_id']).count()[['program_name']].groupby('subsr_id').count().reset_index().rename(columns={'program_name':'시청vod수'}), how='left')

In [193]:
# 사용자별 주별 평균 시청 vod수
user_info = user_info.merge(vod_log.groupby(['subsr_id', 'week', 'program_id']).count()[['program_name']].groupby(['subsr_id', 'week']).count().groupby('subsr_id').mean().reset_index().rename(columns={'program_name':'주평균시청vod수'}), how='left')

In [208]:
# 사용자별 방문빈도수 (day기반)
user_info = user_info.merge(cont_log.groupby(['subsr_id','month', 'day']).count()[['program_id']].groupby(['subsr_id', 'month']).count().groupby('subsr_id').sum([['program_id']]).reset_index().rename(columns={'program_id':'방문빈도수'}), how='left')

In [213]:
# 사용자별 상세보기조회빈도(클릭수)
user_info = user_info.merge(cont_log.groupby(['subsr_id']).count()[['program_id']].reset_index().rename(columns={'program_id':'상세보기조회빈도(클릭수)'}), how='left')

In [215]:
# 사용자별 평균 클릭수 (클릭수 / 방문빈도수)
user_info['평균 클릭수'] = user_info['상세보기조회빈도(클릭수)'] / user_info['방문빈도수']

In [221]:
# 사용자별 시청빈도(시청수)
user_info = user_info.merge(vod_log.groupby(['subsr_id']).count()[['program_id']].reset_index().rename(columns={'program_id':'시청빈도(시청수)'}), how='left')

In [224]:
# 사용자별 평균 시청수 (시청수 / 방문빈도수)
user_info['평균 시청수'] = user_info['시청빈도(시청수)'] / user_info['방문빈도수']

In [230]:
# 사용자별 시청시간합계 (use_tns)
user_info = user_info.merge(vod_log.groupby(['subsr_id']).sum()[['use_tms']].reset_index().rename(columns={'use_tms':'시청시간합계'}), how='left')

In [232]:
# 사용자별 평균 시청시간 (시청시가합계 / 방문빈도수)
user_info['평균 시청시간'] = user_info['시청시간합계'] / user_info['방문빈도수']

In [236]:
# 사용자별 vod 시청 비율 (완전한 시청에 대한 기준?)
vod_log['use_tms_ratio'] = vod_log['use_tms'] / vod_log['disp_rtm_sec']
user_info = user_info.merge(vod_log.groupby(['subsr_id']).mean()[['use_tms_ratio']].reset_index().rename(columns={'use_tms_ratio':'vod평균시청율'}), how='left')

In [245]:
# 시청전환율
user_info['시청전환율'] = user_info['시청vod수'] / user_info['상세보기vod수']

In [254]:
# age_limit (청불 시청 여부) - 근데 이건 부정확할 수 있음
# 근데, 둘다 1이라면, 분리가 필요하다고 말할 수 있을 듯!
user_info.loc[user_info[user_info['subsr_id'].isin(cont_log[cont_log['age_limit']=='청불']['subsr_id'].unique())].index, '청불여부_상세보기'] = 1
user_info['청불여부_상세보기'] = user_info['청불여부_상세보기'].fillna(0)

In [256]:
user_info.loc[user_info[user_info['subsr_id'].isin(vod_log[vod_log['age_limit']=='청불']['subsr_id'].unique())].index, '청불여부_시청'] = 1
user_info['청불여부_시청'] = user_info['청불여부_시청'].fillna(0)

In [268]:
# 타입 설정 (0: 시청기록 x, 1: 시청기록 o)

# 1) 시청목록 있는데 상세보기 없는 유저 - 22명 (이상치라고 했음)
user_info.loc[user_info[(user_info['시청vod수'].isnull())].index, 'user_type'] = 0
user_info['user_type'] = user_info['user_type'].fillna(1)

In [271]:
user_info.to_csv('./EDA/user_info.csv')

In [ ]:
# 키즈는 주로 주말인지? 주중인지?

In [374]:
user_info.loc[user_info[user_info['시청전환율']>1].index, '시청전환율'] = 1

In [376]:
user_info[user_info['user_type']==1][['시청전환율', '시청빈도(시청수)']].describe()

,시청전환율,시청빈도(시청수)
count,385.000000,407.000000
mean,0.460522,18.921376
std,0.284251,53.436752
min,0.020585,1.000000
25%,0.250000,1.500000
50%,0.400000,4.000000
75%,0.571429,13.500000
max,1.000000,588.000000


In [377]:
user_info2 = user_info[user_info['user_type']==1]

In [381]:
user_info2.loc[user_info2[(user_info2['시청전환율'] >= user_info2['시청전환율'].quantile(0.75)) & (user_info2['시청빈도(시청수)'] >= user_info2['시청빈도(시청수)'].quantile(0.75))].index, 'cluster'] = 1
user_info2.loc[user_info2[(user_info2['시청전환율'] >= user_info2['시청전환율'].quantile(0.75)) & (user_info2['시청빈도(시청수)'] < user_info2['시청빈도(시청수)'].quantile(0.75))].index, 'cluster'] = 2
user_info2.loc[user_info2[(user_info2['시청전환율'] < user_info2['시청전환율'].quantile(0.75)) & (user_info2['시청빈도(시청수)'] >= user_info2['시청빈도(시청수)'].quantile(0.75))].index, 'cluster'] = 3
user_info2.loc[user_info2[(user_info2['시청전환율'] < user_info2['시청전환율'].quantile(0.75)) & (user_info2['시청빈도(시청수)'] < user_info2['시청빈도(시청수)'].quantile(0.75))].index, 'cluster'] = 4

In [386]:
user_info2[user_info2['cluster'].notnull()].to_csv('./EDA/user_info2.csv')

In [383]:
user_info2['cluster'].value_counts()

4.0    217
2.0     71
3.0     70
1.0     27
Name: cluster, dtype: int64

### VOD별

In [351]:
vod_info

,program_id,program_name,ct_cl,program_genre,ACTR_DISP,age_limit,상세보기user수,시청user수,주평균시청user수,시청빈도(시청수),상세보기조회빈도(클릭수),시청시간합계,user별평균시청시간,user별평균시청율,시청전환율,kids사용자시청수
0,0,그것이알고싶다,TV 시사/교양,시사교양,김상중,전체,18.0,17.0,4.000000,80.0,147.0,224634.0,13213.764706,0.638160,0.944444,1.0
1,1,꼬리에꼬리를무는그날이야기,TV 시사/교양,"시사교양, 토크","장도연,장현성,장성규",전체,25.0,21.0,6.444444,172.0,295.0,500827.0,23848.904762,0.655602,0.840000,2.0
2,2,인간극장,TV 시사/교양,"시사교양, 인물","이숭배,유정옥",전체,11.0,8.0,1.625000,69.0,78.0,99371.0,12421.375000,0.747369,0.727273,0.0
3,3,오은영 리포트 결혼 지옥,TV 시사/교양,"리얼리티, 토크","오은영,소유진,하하,김응수,박지민",전체,9.0,7.0,2.666667,42.0,82.0,129467.0,18495.285714,0.601827,0.777778,0.0
4,4,실화탐사대,TV 시사/교양,시사교양,"신동엽,서인,강다솜,박지훈",15,9.0,6.0,2.428571,26.0,66.0,51809.0,8634.833333,0.683851,0.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4645,4645,해리포터와죽음의성물 1부,영화,"액션, 판타지, SF, 모험","다니엘 래드클리프,엠마 왓슨,루퍼트 그린트,빌 나이,리차드 그리피스,해리 멜링,줄리...",전체,2.0,1.0,1.000000,1.0,7.0,15.0,15.000000,0.001712,0.500000,1.0
4646,4646,해운대,영화,"드라마, 액션, 모험","설경구,하지원,박중훈,엄정화,이민기,강예원,김인권",12,10.0,8.0,2.000000,11.0,28.0,25340.0,3167.500000,0.319949,0.800000,0.0
4647,4647,해피엔드,영화,"드라마, 멜로, 공포, 스릴러","최민식,전도연,주진모,주현,황미선,김병춘,유연수,허예인",15,54.0,37.0,12.400000,74.0,197.0,48418.0,1308.594595,0.109050,0.685185,0.0
4648,4648,허니,영화,"드라마, 멜로, 판타지, 로맨스","제시카 알바,메키 파이퍼,릴 로메오,조이 브라이언트",12,1.0,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN


In [288]:
# 프로그램별 상세보기 user수
vod_info = vod_info.merge(cont_log.groupby(['program_id', 'subsr_id']).count()[['program_name']].groupby('program_id').count().reset_index().rename(columns={'program_name':'상세보기user수'}), how='left')

In [293]:
# 프로그램별 시청 user수
vod_info = vod_info.merge(vod_log.groupby(['program_id', 'subsr_id']).count()[['program_name']].groupby('program_id').count().reset_index().rename(columns={'program_name':'시청user수'}), how='left')

In [304]:
# 프로그램별 주평균 시청 user 수
vod_info = vod_info.merge(vod_log.groupby(['program_id', 'week', 'subsr_id']).count()[['program_name']].groupby(['program_id', 'week']).count().groupby('program_id').mean().reset_index().rename(columns={'program_name':'주평균시청user수'}), how='left')

In [309]:
# 시청빈도수
vod_info = vod_info.merge(vod_log.groupby(['program_id']).count()[['program_name']].reset_index().rename(columns={'program_name':'시청빈도(시청수)'}), how='left')

In [312]:
# 상세보기 조회빈도(클릭수)
vod_info = vod_info.merge(cont_log.groupby(['program_id']).count()[['program_name']].reset_index().rename(columns={'program_name':'상세보기조회빈도(클릭수)'}), how='left')

In [318]:
# 시청시간 합계
vod_info = vod_info.merge(vod_log.groupby(['program_id']).sum()[['use_tms']].reset_index().rename(columns={'use_tms':'시청시간합계'}), how='left')

In [324]:
# user별 평균 시청시간
vod_info = vod_info.merge(vod_log.groupby(['program_id', 'subsr_id']).sum()[['use_tms']].groupby('program_id').mean().reset_index().rename(columns={'use_tms':'user별평균시청시간'}), how='left')

In [329]:
# 평균 시청율 (use_tms_ratio)
vod_log['use_tms_ratio'] = vod_log['use_tms'] / vod_log['disp_rtm_sec']
vod_info = vod_info.merge(vod_log.groupby(['program_id']).mean()[['use_tms_ratio']].reset_index().rename(columns={'use_tms_ratio':'user별평균시청율'}), how='left')

In [332]:
# 시청전환율
vod_info['시청전환율'] = vod_info['시청user수'] /  vod_info['상세보기user수']

In [350]:
# kids = 1인 사람의 시청수
vod_info = vod_info.merge(vod_log.merge(users[['subsr_id', 'kids']], how='left').groupby(['program_id', 'subsr_id']).max()[['kids']].groupby('program_id').sum().reset_index().rename(columns={'kids':'kids사용자시청수'}), how='left')

In [353]:
vod_info.to_csv('./EDA/vod_info.csv')

In [355]:
vod_info

,program_id,program_name,ct_cl,program_genre,ACTR_DISP,age_limit,상세보기user수,시청user수,주평균시청user수,시청빈도(시청수),상세보기조회빈도(클릭수),시청시간합계,user별평균시청시간,user별평균시청율,시청전환율,kids사용자시청수
0,0,그것이알고싶다,TV 시사/교양,시사교양,김상중,전체,18.0,17.0,4.000000,80.0,147.0,224634.0,13213.764706,0.638160,0.944444,1.0
1,1,꼬리에꼬리를무는그날이야기,TV 시사/교양,"시사교양, 토크","장도연,장현성,장성규",전체,25.0,21.0,6.444444,172.0,295.0,500827.0,23848.904762,0.655602,0.840000,2.0
2,2,인간극장,TV 시사/교양,"시사교양, 인물","이숭배,유정옥",전체,11.0,8.0,1.625000,69.0,78.0,99371.0,12421.375000,0.747369,0.727273,0.0
3,3,오은영 리포트 결혼 지옥,TV 시사/교양,"리얼리티, 토크","오은영,소유진,하하,김응수,박지민",전체,9.0,7.0,2.666667,42.0,82.0,129467.0,18495.285714,0.601827,0.777778,0.0
4,4,실화탐사대,TV 시사/교양,시사교양,"신동엽,서인,강다솜,박지훈",15,9.0,6.0,2.428571,26.0,66.0,51809.0,8634.833333,0.683851,0.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4645,4645,해리포터와죽음의성물 1부,영화,"액션, 판타지, SF, 모험","다니엘 래드클리프,엠마 왓슨,루퍼트 그린트,빌 나이,리차드 그리피스,해리 멜링,줄리...",전체,2.0,1.0,1.000000,1.0,7.0,15.0,15.000000,0.001712,0.500000,1.0
4646,4646,해운대,영화,"드라마, 액션, 모험","설경구,하지원,박중훈,엄정화,이민기,강예원,김인권",12,10.0,8.0,2.000000,11.0,28.0,25340.0,3167.500000,0.319949,0.800000,0.0
4647,4647,해피엔드,영화,"드라마, 멜로, 공포, 스릴러","최민식,전도연,주진모,주현,황미선,김병춘,유연수,허예인",15,54.0,37.0,12.400000,74.0,197.0,48418.0,1308.594595,0.109050,0.685185,0.0
4648,4648,허니,영화,"드라마, 멜로, 판타지, 로맨스","제시카 알바,메키 파이퍼,릴 로메오,조이 브라이언트",12,1.0,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN


### 성인 시청 로그 현황

In [366]:
vod_log.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7701 entries, 0 to 7723
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   subsr_id       7701 non-null   int64         
 1   program_id     7701 non-null   object        
 2   program_name   7701 non-null   object        
 3   episode_num    7701 non-null   int64         
 4   ct_cl          7701 non-null   object        
 5   program_genre  7701 non-null   object        
 6   log_dt         7701 non-null   datetime64[ns]
 7   month          7701 non-null   int64         
 8   week           7701 non-null   int64         
 9   day            7701 non-null   int64         
 10  hour           7701 non-null   int64         
 11  minute         7701 non-null   int64         
 12  weekday        7701 non-null   int64         
 13  day_name       7701 non-null   object        
 14  isholiday      7701 non-null   float64       
 15  isweekend      7701 n

In [ ]:
vod_log['age_limit']

In [389]:
vod_info.groupby(['ct_cl']).count()[['program_id']]

,program_id
ct_cl,
TV 시사/교양,125
TV 연예/오락,288
TV드라마,908
TV애니메이션,295
공연/음악,1
다큐,31
라이프,7
스포츠,5
영화,2752
